# Long-term Memory: Cross-Session Knowledge

## Introduction

In this notebook, you'll learn about long-term memory - persistent knowledge that survives across sessions. While working memory handles the current conversation, long-term memory stores important facts, preferences, and experiences that should be remembered indefinitely.

### What You'll Learn

- What long-term memory is and why it's essential
- The three types of long-term memories: semantic, episodic, and message
- How to store and retrieve long-term memories
- How semantic search works with memories
- How automatic deduplication prevents redundancy

### Prerequisites

- Completed Section 2 notebooks
- Completed `01_working_memory_with_extraction_strategies.ipynb`
- Redis 8 running locally
- Agent Memory Server running
- OpenAI API key set

## Concepts: Long-term Memory

### What is Long-term Memory?

Long-term memory is **persistent, cross-session knowledge** about users, preferences, and important facts. Unlike working memory (which is session-scoped), long-term memory:

- ✅ Survives across sessions
- ✅ Accessible from any conversation
- ✅ Searchable via semantic vector search
- ✅ Automatically deduplicated
- ✅ Organized by user/namespace

### Working Memory vs. Long-term Memory

| Working Memory | Long-term Memory |
|----------------|------------------|
| **Session-scoped** | **User-scoped** |
| Current conversation | Important facts |
| TTL-based (expires) | Persistent |
| Full message history | Extracted knowledge |
| Loaded/saved each turn | Searched when needed |

### Three Types of Long-term Memories

The Agent Memory Server supports three types of long-term memories:

1. **Semantic Memory** - Facts and knowledge
   - Example: "Student prefers online courses"
   - Example: "Student's major is Computer Science"
   - Example: "Student wants to graduate in 2026"

2. **Episodic Memory** - Events and experiences
   - Example: "Student enrolled in CS101 on 2024-09-15"
   - Example: "Student asked about machine learning on 2024-09-20"
   - Example: "Student completed Data Structures course"

3. **Message Memory** - Important conversation snippets
   - Example: Full conversation about career goals
   - Example: Detailed discussion about course preferences

## Choosing the Right Memory Type

Understanding WHEN to use each memory type is crucial for effective memory management.

### Decision Framework

#### Use Semantic Memory for: Facts and Preferences

**Characteristics:**
- Timeless information (not tied to specific moment)
- Likely to be referenced repeatedly
- Can be stated independently of context

**Examples:**
```python
# ✅ Good semantic memories
"Student prefers online courses"
"Student's major is Computer Science"  
"Student wants to graduate in 2026"
"Student struggles with mathematics"
"Student is interested in machine learning"
```

**Why semantic:**
- Facts that don't change often
- Will be useful across many sessions
- Don't need temporal context

---

#### Use Episodic Memory for: Events and Timeline

**Characteristics:**
- Time-bound events
- Sequence/timeline matters
- Tracking progress or history

**Examples:**
```python
# ✅ Good episodic memories
"Student enrolled in CS101 on 2024-09-15"
"Student completed CS101 on 2024-12-10"
"Student started CS201 on 2024-01-15"
"Student asked about career planning on 2024-10-20"
"Student expressed concerns about workload on 2024-10-27"
```

**Why episodic:**
- Events have specific dates
- Order of events matters (CS101 before CS201)
- Tracking student's journey over time

---

#### Use Message Memory for: Context-Rich Conversations

**Characteristics:**
- Full context is crucial
- Tone/emotion matters
- May need exact wording
- Complex multi-part discussions

**Examples:**
```python
# ✅ Good message memories
"Detailed career planning discussion: [full conversation]"
"Professor's specific advice about research opportunities: [full message]"
"Student's explanation of personal learning challenges: [full message]"
```

**Why message:**
- Summary would lose important nuance
- Context around the words matters
- Verbatim quote may be needed

**⚠️ Use sparingly - message memories are token-expensive!**

### Examples: Right vs. Wrong

#### Scenario 1: Student States Preference

**User says:** "I prefer online courses because I work during the day."

❌ **Wrong:**
```python
# Message memory (too verbose)
memory = "Student said: 'I prefer online courses because I work during the day.'"
```

✅ **Right:**
```python
# Semantic memories (extracted facts)
memory1 = "Student prefers online courses"
memory2 = "Student works during the day"
```

**Why:** Simple facts don't need full verbatim storage.

---

#### Scenario 2: Course Completion

**User says:** "I just finished CS101 last week!"

❌ **Wrong:**
```python
# Semantic (loses temporal context)
memory = "Student completed CS101"
```

✅ **Right:**
```python
# Episodic (preserves timeline)
memory = "Student completed CS101 on 2024-10-20"
```

**Why:** Timeline matters for prerequisites and planning.

---

#### Scenario 3: Complex Career Advice

**Conversation:** 20-message discussion about career path, including professor's nuanced advice about research vs. industry, timing of applications, and specific companies to target.

❌ **Wrong:**
```python
# Semantic (loses too much)
memory = "Student discussed career planning"
```

✅ **Right:**
```python
# Message memory (preserves context)
memory = [Full conversation thread with all nuance]
```

**Why:** Details and context are critical, summary inadequate.

### Quick Reference Table

| Information Type | Memory Type | Example |
|-----------------|-------------|----------|
| Preference | Semantic | "Prefers morning classes" |
| Fact | Semantic | "Major is Computer Science" |
| Goal | Semantic | "Wants to graduate in 2026" |
| Event | Episodic | "Enrolled in CS401 on 2024-09-15" |
| Timeline | Episodic | "Completed CS101, then CS201" |
| Progress | Episodic | "Asked about ML three times" |
| Complex discussion | Message | [Full career planning conversation] |
| Nuanced advice | Message | [Professor's detailed guidance] |

### Default Strategy: Prefer Semantic

**When in doubt:**
1. Can you extract a simple fact? → **Semantic**
2. Is timing important? → **Episodic**
3. Is full context crucial? → **Message** (use rarely)

**Most memories should be semantic** - they're compact, searchable, and efficient.

### How Semantic Search Works

Long-term memories are stored with vector embeddings, enabling semantic search:

- Query: "What does the student like?"
- Finds: "Student prefers online courses", "Student enjoys programming"
- Even though exact words don't match!

### Automatic Deduplication

The Agent Memory Server automatically prevents duplicate memories:

- **Hash-based**: Exact duplicates are rejected
- **Semantic**: Similar memories are merged
- Keeps memory storage efficient

## Setup

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify required environment variables are set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY not found. Please create a .env file with your OpenAI API key. "
        "See SETUP.md for instructions."
    )

print("✅ Environment variables loaded")

In [ ]:
import asyncio
from datetime import datetime
from agent_memory_client import MemoryAPIClient as MemoryClient, MemoryClientConfig
from agent_memory_client.models import ClientMemoryRecord
from agent_memory_client.filters import MemoryType

# Initialize memory client
student_id = "student_123"
config = MemoryClientConfig(
    base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
    default_namespace="redis_university"
)
memory_client = MemoryClient(config=config)

print(f"✅ Memory client initialized for {student_id}")

## Hands-on: Working with Long-term Memory

### Example 1: Storing Semantic Memories (Facts)

Let's store some facts about the student.

In [ ]:
# Store student preferences
await memory_client.create_long_term_memory([ClientMemoryRecord(
    text="Student prefers online courses over in-person classes",
    memory_type="semantic",
    topics=["preferences", "course_format"]
)])

await memory_client.create_long_term_memory([ClientMemoryRecord(
    text="Student's major is Computer Science with a focus on AI/ML",
    memory_type="semantic",
    topics=["academic_info", "major"]
)])

await memory_client.create_long_term_memory([ClientMemoryRecord(
    text="Student wants to graduate in Spring 2026",
    memory_type="semantic",
    topics=["goals", "graduation"]
)])

await memory_client.create_long_term_memory([ClientMemoryRecord(
    text="Student prefers morning classes, no classes on Fridays",
    memory_type="semantic",
    topics=["preferences", "schedule"]
)])

print("✅ Stored 4 semantic memories (facts about the student)")

### Example 2: Storing Episodic Memories (Events)

Let's store some events and experiences.

In [ ]:
# Store course enrollment events
await memory_client.create_long_term_memory([ClientMemoryRecord(
    text="Student enrolled in CS101: Introduction to Programming on 2024-09-01",
    memory_type="episodic",
    topics=["enrollment", "courses", "CS101"]
)])

await memory_client.create_long_term_memory([ClientMemoryRecord(
    text="Student completed CS101 with grade A on 2024-12-15",
    memory_type="episodic",
    topics=["completion", "grades", "CS101"]
)])

await memory_client.create_long_term_memory([ClientMemoryRecord(
    text="Student asked about machine learning courses on 2024-09-20",
    memory_type="episodic",
    topics=["inquiry", "machine_learning"]
)])

print("✅ Stored 3 episodic memories (events and experiences)")

### Example 3: Searching Memories with Semantic Search

Now let's search for memories using natural language queries.

In [ ]:
# Search for preferences
print("Query: 'What does the student prefer?'\n")
results = await memory_client.search_long_term_memory(
    text="What does the student prefer?",
    limit=3
)

for i, memory in enumerate(results.memories, 1):
    print(f"{i}. {memory.text}")
    print(f"   Type: {memory.memory_type} | Topics: {', '.join(memory.topics)}")
    print()

In [ ]:
# Search for academic information
print("Query: 'What is the student studying?'\n")
results = await memory_client.search_long_term_memory(
    text="What is the student studying?",
    limit=3
)

for i, memory in enumerate(results.memories, 1):
    print(f"{i}. {memory.text}")
    print(f"   Type: {memory.memory_type}")
    print()

In [ ]:
# Search for course history
print("Query: 'What courses has the student taken?'\n")
results = await memory_client.search_long_term_memory(
    text="What courses has the student taken?",
    limit=3
)

for i, memory in enumerate(results.memories, 1):
    print(f"{i}. {memory.text}")
    print(f"   Type: {memory.memory_type} | Topics: {', '.join(memory.topics or [])}")
    print()

### Example 4: Demonstrating Deduplication

Let's try to store duplicate memories and see how deduplication works.

In [ ]:
# Try to store an exact duplicate
print("Attempting to store exact duplicate...")
try:
    await memory_client.create_long_term_memory([ClientMemoryRecord(
        text="Student prefers online courses over in-person classes",
        memory_type="semantic",
        topics=["preferences", "course_format"]
)])
    print("❌ Duplicate was stored (unexpected)")
except Exception as e:
    print(f"✅ Duplicate rejected: {e}")

# Try to store a semantically similar memory
print("\nAttempting to store semantically similar memory...")
try:
    await memory_client.create_long_term_memory([ClientMemoryRecord(
        text="Student likes taking classes online instead of on campus",
        memory_type="semantic",
        topics=["preferences", "course_format"]
)])
    print("Memory stored (may be merged with existing similar memory)")
except Exception as e:
    print(f"✅ Similar memory rejected: {e}")

### Example 5: Cross-Session Memory Access

Let's simulate a new session and show that memories persist.

In [ ]:
# Create a new memory client (simulating a new session)
config = MemoryClientConfig(
    base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
    default_namespace="redis_university"
)
new_session_client = MemoryClient(config=config)

print("New session started for the same student\n")

# Search for memories from the new session
print("Query: 'What do I prefer?'\n")
results = await new_session_client.search_long_term_memory(
    text="What do I prefer?",
    limit=3
)

print("✅ Memories accessible from new session:\n")
for i, memory in enumerate(results.memories, 1):
    print(f"{i}. {memory.text}")
    print()


### Example 6: Filtering by Memory Type and Topics

In [ ]:
# Get all semantic memories
print("All semantic memories (facts):\n")
results = await memory_client.search_long_term_memory(
    text="",  # Empty query returns all
    memory_type=MemoryType(eq="semantic"),
    limit=10
)

for i, memory in enumerate(results.memories, 1):
    print(f"{i}. {memory.text}")
    print(f"   Topics: {', '.join(memory.topics)}")
    print()

In [ ]:
# Get all episodic memories
print("All episodic memories (events):\n")
results = await memory_client.search_long_term_memory(
    text="",
    memory_type=MemoryType(eq="episodic"),
    limit=10
)

for i, memory in enumerate(results.memories, 1):
    print(f"{i}. {memory.text}")
    print(f"   Topics: {', '.join(memory.topics or [])}")
    print()

## Key Takeaways

### When to Use Long-term Memory

Store in long-term memory:
- ✅ User preferences and settings
- ✅ Important facts about the user
- ✅ Goals and objectives
- ✅ Significant events and milestones
- ✅ Completed courses and achievements

Don't store in long-term memory:
- ❌ Temporary conversation context
- ❌ Trivial details
- ❌ Information that changes frequently
- ❌ Sensitive data without proper handling

### Memory Types Guide

**Semantic (Facts):**
- "Student prefers X"
- "Student's major is Y"
- "Student wants to Z"

**Episodic (Events):**
- "Student enrolled in X on DATE"
- "Student completed Y with grade Z"
- "Student asked about X on DATE"

**Message (Conversations):**
- Important conversation snippets
- Detailed discussions worth preserving

### Best Practices

1. **Use descriptive topics** - Makes filtering and categorization easier
2. **Write clear memory text** - Will be searched semantically
3. **Include relevant details in text** - Dates, names, and context help with retrieval
4. **Let deduplication work** - Don't worry about duplicates
5. **Search before storing** - Check if similar memory exists

## Exercises

1. **Store your own memories**: Create 5 semantic and 3 episodic memories about a fictional student. Search for them.

2. **Test semantic search**: Create memories with different wordings but similar meanings. Search with various queries to see what matches.

3. **Explore topics**: Add rich topics to episodic memories. How can you use topic filtering in your agent?

4. **Cross-session test**: Create a memory, close the notebook, restart, and verify the memory persists.

## Summary

In this notebook, you learned:

- ✅ Long-term memory stores persistent, cross-session knowledge
- ✅ Three types: semantic (facts), episodic (events), message (conversations)
- ✅ Semantic search enables natural language queries
- ✅ Automatic deduplication prevents redundancy
- ✅ Memories are user-scoped and accessible from any session

**Next:** In the next notebook, we'll integrate working memory and long-term memory to build a complete memory system for our agent.

## Memory Lifecycle and Persistence

Understanding how long memories last and when they expire is important for managing your agent's memory system.

### Working Memory Lifecycle

**TTL (Time To Live): 24 hours by default**

```
Session Created
    ↓
Messages Stored (each turn adds messages)
    ↓
[24 hours of inactivity]
    ↓
Working Memory Automatically Expires ❌
```

**What this means:**
- ✅ Working memory lasts for the duration of active conversation
- ✅ Plus 24 hours after last activity
- ✅ Automatically cleaned up (no action needed)
- ⚠️ After expiration, conversation context is lost

**Example Timeline:**
```
10:00 AM - Session starts
10:15 AM - User asks about CS401
10:20 AM - User asks about prerequisites
10:25 AM - Session ends (user leaves)

[24 hours later]
10:25 AM next day - Working memory still available ✅
10:26 AM next day - Working memory expires ❌

If user returns:
10:30 AM next day - New session starts (no previous context) 🆕
```

### Long-term Memory Lifecycle

**Persistence: Indefinite (no automatic expiration)**

```
Memory Created
    ↓
Stored in Long-term Memory
    ↓
Available Across All Sessions ✅
    ↓
Persists Until Manually Deleted
```

**What this means:**
- ✅ Long-term memories never automatically expire
- ✅ Available across all sessions (any time user returns)
- ✅ Survives working memory expiration
- ⚠️ Must be manually deleted if needed

**Example:**
```
Day 1, Session 1:
- User: "I prefer online courses"
- Extracted to long-term memory: "Student prefers online courses"

Day 2, Session 2 (different session):
- Long-term memory retrieved: "Student prefers online courses" ✅
- Working memory from Day 1: Expired ❌

Day 30, Session 10:
- Long-term memory still available: "Student prefers online courses" ✅
```

### Why This Design?

**Working Memory = Short-term Context**
- Conversation-specific
- High detail (full messages)
- Expires to save storage
- Like human short-term memory

**Long-term Memory = Persistent Facts**
- User-specific knowledge
- Important facts only
- Persists indefinitely
- Like human long-term memory

### Important Implications

#### 1. Extract Before Expiration

**Working memory expires in 24 hours!**

```python
# ✅ Good: Extraction happens automatically
# Agent Memory Server extracts facts from working memory
# BEFORE it expires

# ❌ Bad: Don't rely on working memory persisting
# It will expire and take conversation context with it
```

**The Agent Memory Server handles extraction automatically** - this is why we use it!

#### 2. Long-term Memories Are Permanent

**Unless you explicitly delete them:**

```python
# Manual deletion (when needed)
await memory_client.delete_memory(memory_id)

# Or delete all memories for a user
await memory_client.delete_all_user_memories(user_id)
```

**Use cases for deletion:**
- User requests deletion
- Memory becomes outdated (preference changed)
- Incorrect information was stored

### Practical Example: Multi-Day Conversation

**Day 1 (Session 1):**
```python
User: "I'm interested in machine learning"
Agent: [Responds]
Working Memory: [Full conversation]
Long-term: "Student interested in machine learning" (extracted)
```

**Day 2 (Session 2, 30 hours later):**
```python
# Working memory from Day 1: EXPIRED ❌
# Long-term memory: Still available ✅

User: "What ML courses do you recommend?"
Agent retrieves long-term: "Student interested in machine learning"
Agent: [Makes relevant recommendations using stored fact]
```

**Agent remembers across sessions thanks to long-term memory!**

### Best Practices

1. **Trust the extraction process**
   - Agent Memory Server automatically extracts important facts
   - Happens in background during conversation
   - Important info moves to long-term before expiration

2. **Don't worry about working memory expiration**
   - It's designed to expire
   - Important facts are already extracted
   - New sessions get clean slate

3. **Long-term memories are your persistent knowledge**
   - Think of them as "what the agent knows about the user"
   - Cross-session, cross-conversation
   - The foundation of personalization

4. **Clean up when needed**
   - Outdated preferences (user says "I now prefer in-person classes")
   - Incorrect information (wrong major was recorded)
   - User requests deletion

### Summary

| Memory Type | Duration | Cleanup | Purpose |
|-------------|----------|---------|----------|
| Working | 24 hours | Automatic | Current conversation |
| Long-term | Indefinite | Manual | Persistent knowledge |

**Working memory is temporary context. Long-term memory is permanent knowledge.**

Understanding this distinction helps you design better memory strategies.